# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [13]:
# all import statements needed for the project, for example:

import math
import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
import re
import random
import geopandas
import numpy as np

In [2]:
# any general notebook setup, like log formatting

In [9]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [ ] Taxi data:
    * [ ] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [ ] Load the data using `pandas`
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

### Calculating distance
_**TODO:** Write some prose that tells the reader what you're about to do here._

In [4]:
### Define a function to calculate the distance between two locations by their coordinates
def calculate_distance(from_coord, to_coord):
    dlat = to_coord[0] - from_coord[0]
    dlon = to_coord[1] - from_coord[1]
    a = (math.sin(dlat/2))**2 + math.cos(from_coord[0]) * math.cos(to_coord[0]) * (math.sin(dlon/2))**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = 6371 * c
    return distance

In [ ]:
def add_distance_column(dataframe):
    #add a distance column to both uber and taxi files
    dis = []
    for i in range(len(dataframe.index)):
        dis.append(calculate_distance((dataframe.iloc[i]['PU_lat'],dataframe.iloc[i]['PU_lon']), (dataframe.iloc[i]['DO_lat'],dataframe.iloc[i]['DO_lon'])))
    
    dataframe.insert(len(dataframe.columns), "distance", dis)

### Processing Taxi Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [7]:
#Function to find all parquet files of yellow cab racords on the website
def find_taxi_csv_urls():
    response = requests.get(TAXI_URL)
    soup = bs4.BeautifulSoup(response.content, 'html.parser')
    yellow_a_tags = soup.find_all("a", attrs={"title": "Yellow Taxi Trip Records"})
    yellow_taxis = [a["href"] for a in yellow_a_tags] # all yellow taxi trip records urls
    
    # creating a pattern to filter url for desired time frame from January 2009 to June 2015
    pattern1 = re.compile(r"yellow_tripdata_2009-([0-9]{2}).parquet")
    pattern2 = re.compile(r"yellow_tripdata_201([0-4]{1})-([0-9]{2}).parquet")
    pattern3 = re.compile(r"yellow_tripdata_2015-0([1-6]{1}).parquet")
    
    filtered_taxi_files = []
    
    #finding all yellow taxi record urls from January 2009 to June 2015
    for link in yellow_taxis:
        match = pattern1.search(link)
        if match:
            filtered_taxi_files.append(match.string)
    
    for link in yellow_taxis:
        match = pattern2.search(link)
        if match:
            filtered_taxi_files.append(match.string)
    
    for link in yellow_taxis:
        match = pattern3.search(link)
        if match:
            filtered_taxi_files.append(match.string)
    
    return filtered_taxi_files

In [ ]:
#read the taxi_zones geo files and extract the lat/log of the center point of each polygon
taxi_zones = "taxi_zones.shp"
zones = geopandas.read_file(taxi_zones)
zones = zones.to_crs(4326)
zones["lon"] = zones.centroid.x
zones["lat"] = zones.centroid.y
latitudes = zones.set_index('LocationID').to_dict()['lat']
longitudes = zones.set_index('LocationID').to_dict()['lon']

In [ ]:
def get_and_clean_month_taxi_data(url):
   
    #download and read url into dataframe
    r = requests.get(url, stream = [True])
    file_name = ''.join(c for c in url if c.isdigit()) + "_yellowtaxi"
    with open(file_name, 'wb') as f:
        f.write(r.content)
    df = pd.read_parquet(file_name)
    
    if "PULocationID" in df.columns:
        df.insert(9, "PU_lon", df["PULocationID"].map(longitudes))
        df.insert(10, "PU_lat", df["PULocationID"].map(latitudes))
        df.insert(11, "DO_lon",df["DOLocationID"].map(longitudes))
        df.insert(12, "DO_lat", df["DOLocationID"].map(latitudes))
    
    #delete the unwanted columns
    if "vendor_id" in df.columns:
        df.drop(['vendor_id', 'passenger_count', 'trip_distance', 'rate_code', 'store_and_fwd_flag', 'payment_type', 'fare_amount', 'surcharge', 'mta_tax','tolls_amount'], axis=1, inplace=True)
        df = df.rename(columns = {
            'pickup_datetime' : 'PU_datetime',
            'dropoff_datetime' : 'DO_datetime',
            'pickup_longitude' : 'PU_lon',
            'pickup_latitude' : 'PU_lat',
            'dropoff_longitude' : 'DO_lon',
            'dropoff_latitude' : 'DO_lat',
            'tip_amount' : 'tip_amt',
            'total_amount' : 'total_amt', 
        })
    if "vendor_name" in df.columns:
        df.drop(['vendor_name', 'Passenger_Count', 'Trip_Distance', 'Rate_Code', 'store_and_forward', 'Payment_Type', 'Fare_Amt', 'surcharge', 'mta_tax', 'Tolls_Amt' ], axis=1, inplace=True)
        df = df.rename(columns = {
            'Trip_Pickup_DateTime' : 'PU_datetime',
            'Trip_Dropoff_DateTime' : 'DO_datetime',
            'Start_Lon' : 'PU_lon',
            'Start_Lat' : 'PU_lat',
            'End_Lon' : 'DO_lon',
            'End_Lat' : 'DO_lat',
            'Tip_Amt' : 'tip_amt',
            'Total_Amt' : 'total_amt', 
        })
    if "VendorID" in df.columns:
        df.drop(['VendorID', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'airport_fee'], axis=1, inplace=True)
        df = df.rename(columns = {
            'tpep_pickup_datetime' : 'PU_datetime',
            'tpep_dropoff_datetime' : 'DO_datetime',
            'tip_amount' : 'tip_amt',
            'total_amount' : 'total_amt', 
        })
        
    # delete any rows with null input
    df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
    
    # filter data to be within desire pick-up and drop-off locations
    df = df[(df['PU_lat']>40.560445) & (df['PU_lat']<40.908524)]
    df = df[(df['DO_lat']>40.560445) & (df['DO_lat']<40.908524)]
    df = df[(df['PU_lon']> -74.242330) & (df['PU_lon']< -73.717047)]
    df = df[(df['DO_lon']> -74.242330) & (df['DO_lon']< -73.717047)]
    
    
    #sample 2500 rows from each file
    df_sample = df.sample(2500)
    
    return df_sample

In [ ]:
def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    
    all_csv_urls = find_taxi_csv_urls()
    for csv_url in all_csv_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month_taxi_data(csv_url)
        add_distance_column(dataframe)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes)
    return taxi_data

### Processing Uber Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [10]:
def load_and_clean_uber_data(csv_file):
    df = pd.read_csv(csv_file)
    df = df[~(df == 0).any(axis=1)]
    df = df[(df['pickup_latitude']>40.560445) & (df['pickup_latitude']<40.908524)]
    df = df[(df['dropoff_latitude']>40.560445) & (df['dropoff_latitude']<40.908524)]
    df = df[(df['pickup_longitude']> -74.242330) & (df['pickup_longitude']< -73.717047)]
    df = df[(df['dropoff_longitude']> -74.242330) & (df['dropoff_longitude']< -73.717047)]
    df = df.rename(columns = {
            'pickup_datetime' : 'PU_datetime',
            'pickup_longitude' : 'PU_lon',
            'pickup_latitude' : 'PU_lat',
            'dropoff_longitude' : 'DO_lon',
            'dropoff_latitude' : 'DO_lat',
        })
    del df['key']
    del df['Unnamed: 0']
    return df

In [ ]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_CSV)
    add_distance_column(uber_dataframe)
    return uber_dataframe

### Processing Weather Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [11]:
def clean_month_weather_data_hourly(csv_file):
    df = pd.read_csv(csv_file, low_memory=False)
    df['DATE'] = pd.to_datetime(df['DATE'])
    df = df[['DATE', 'HourlyDewPointTemperature', 'HourlyPrecipitation', 'HourlyWindSpeed']]
    df.replace("T", 0.00, inplace=True)
    df.replace(0.00, np.nan, inplace=True)
    df.dropna(how ="any", inplace = True)
    return df

In [ ]:
def clean_month_weather_data_daily(csv_file):
    df = pd.read_csv(csv_file, low_memory=False)
    df['DATE'] = pd.to_datetime(df['DATE'])
    df = df[['DATE','HourlyPrecipitation', 'DailyAverageDewPointTemperature', 'DailyAverageWindSpeed','DailyPrecipitation']]
    df.replace("T", 0, inplace=True)
    df['HourlyPrecipitation'] = df['HourlyPrecipitation'].str.rstrip('s')
    df['HourlyPrecipitation'] = pd.to_numeric(df['HourlyPrecipitation'])
    df = df.groupby([df['DATE'].dt.date])['HourlyPrecipitation'].sum()

    df1 = pd.read_csv(csv_file, low_memory=False)
    df1['DATE'] = pd.to_datetime(df1['DATE'])
    df1 = df1[['DATE','DailyAverageDewPointTemperature', 'DailyAverageWindSpeed']]
    df1 = df1.groupby([df1['DATE'].dt.date]).sum()
    
    
    frame = [df, df1]
    df2 = pd.concat(frame, axis = 1)
    df2 = df2.rename(columns = {'HourlyPrecipitation' : "DailyPrecipitation"})
    
    return df2

In [ ]:
def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    weather_csv_files = ["2009_weather.csv", "2010_weather.csv", "2011_weather.csv", "2012_weather.csv", "2013_weather.csv", 
                        "2014_weather.csv", "2015_weather.csv"]
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

### Process All Data

_This is where you can actually execute all the required functions._

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
taxi_data = get_and_clean_taxi_data()
uber_data = get_uber_data()
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [14]:
engine = db.create_engine(DATABASE_URL)

In [15]:
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather(
   hourly_weatherId INTEGER PRIMARY KEY AUTOINCREMENT,
   DATE DATE,
   HourlyDewPointTemperature FLOAT,
   HourlyPrecipitation FLOAT
   HourlyWindSpeed FLOAT
)
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather(
   daily_weatherId INTEGER PRIMARY KEY AUTOINCREMENT,
   DATE DATE,
   DailyPrecipitation FLOAT,
   DailyAverageDewPointTemperature FLOAT
   DailyAverageWindSpeed FLOAT
)
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips(
   taxi_tripsId INTEGER PRIMARY KEY AUTOINCREMENT,
   PU_datetime DATE,
   DO_datetime DATE,
   PU_lon FLOAT
   PU_lat FLOAT
   DO_lon FLOAT
   DO_lat FLOAT
   tip_amt FLOAT
   total_amt FLOAT
   distance FLOAT
)
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips(
   uber_tripsId INTEGER PRIMARY KEY AUTOINCREMENT,
   fare_amount FLOAT
   PU_datetime DATE,
   PU_lon FLOAT
   PU_lat FLOAT
   DO_lon FLOAT
   DO_lat FLOAT
   passenger_count INTEGER
   distance FLOAT
)
"""


In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [16]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [17]:
def write_dataframes_to_table(table_to_df_dict):
    for table_name, df in table_to_df_dict.items():
        df.to_sql(table_name, engine, if_exists='append', index=False)

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [18]:
def write_query_to_file(query, outfile):
    with open(outfile, "w") as f:
        f.write(query)

### Query 1

_**TODO:** For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins._


In [ ]:
## most popular hour of the day to take a yellow taxi
QUERY_1 = """
SELECT strftime('%H', PU_datetime) AS hour, COUNT(*) AS num
FROM taxi_trips
GROUP BY hour

ORDER BY num DESC
"""

In [ ]:
engine.execute(QUERY_1).fetchall()

In [ ]:
write_query_to_file(QUERY_1, "most_popular_taxi_hour.sql")

### Query 2

_**TODO:** For the same time frame, what day of the week was the most popular to take an uber?_


In [ ]:
QUERY_2 = """
SELECT strftime('%w', PU_datetime) As day, COUNT(*) AS num
FROM uber_trips
GROUP BY day

ORDER BY num DESC;
"""

In [ ]:
engine.execute(QUERY_2).fetchall()

In [ ]:
write_query_to_file(QUERY_2, "most_popular_uber_day.sql")

### Query 3

_**TODO:** What is the 95% percentile of distance traveled for all hired trips during July 2013?_

In [ ]:
## the 95% percentile of distance traveled for all hired trips during July 2013
QUERY_3 = """
WITH all_hired_trips AS(
SELECT PU_datetime, distance FROM taxi_trips 
WHERE PU_datetime BETWEEN "2013-07-01" AND "2013-07-31"
UNION ALL
SELECT PU_datetime, distance FROM uber_trips WHERE PU_datetime BETWEEN "2013-07-01" AND "2013-07-31")

SELECT distance FROM all_hired_trips
ORDER BY distance ASC

LIMIT 1
OFFSET (SELECT COUNT(*) FROM all_hired_trips) * 95 / 100 ;
"""


In [ ]:
engine.execute(QUERY_3).fetchall()

In [ ]:
write_query_to_file(QUERY_3, "95th Percentile of distance Jul13.sql")

### Query 4

_**TODO:** What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?_

In [ ]:
#the top 10 days with the highest number of hired rides for 2009, and the average distance for each day?
QUERY_4 = """WITH all_hired_trips AS (
SELECT PU_datetime, distance FROM taxi_trips 
WHERE PU_datetime BETWEEN '2009-01-01' AND '2009-12-31'
UNION ALL
SELECT PU_datetime,distance FROM uber_trips WHERE PU_datetime BETWEEN '2009-01-01' AND '2009-12-31')

SELECT PU_datetime AS date, AVG(distance) AS avg_dist, COUNT(*) AS num
FROM all_hired_trips

GROUP BY date
ORDER BY num DESC

LIMIT 10;
"""

In [ ]:
engine.execute(QUERY_4).fetchall()

In [ ]:
write_query_to_file(QUERY_4, "top 10 most rides 2009.sql")

### Query 5

_**TODO:** Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?_

In [ ]:
#top 10 windest days in 2014, and number of hired trips on those days
QUERY_5 ="""
SELECT date(PU_datetime) AS date, COUNT(*) AS num

FROM(SELECT PU_datetime FROM taxi_trips
UNION ALL
SELECT PU_datetime FROM uber_trips)

GROUP BY date
HAVING date IN (SELECT DATE FROM daily_weather WHERE DATE BETWEEN '2014-01-01' AND '2014-12-31'
ORDER BY DailyAverageWindSpeed DESC)

LIMIT 10
"""

In [ ]:
engine.execute(QUERY_5).fetchall()

In [ ]:
write_query_to_file(QUERY_4, "top 10 windest day 2014.sql")

### Query 6

_**TODO:** During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?_

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [19]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_n():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplemented()

In [ ]:
some_dataframe = get_data_for_visual_n()
plot_visual_n(some_dataframe)